In [ ]:
# notebook to obtain results for Figure C1 of https://arxiv.org/pdf/2508.05390v1

In [ ]:
import numpy as np
from pyscf import gto, scf, lo
from pyscf.tools import cubegen

theta_deg = 0
Ht = np.array([0.0000,	0.9289,	1.2321])
Ct = np.array([0.0000,	0.0000,	0.6695])
C1 = np.array([0.0000,	0.0000,	-0.6695])
H1 = np.array([0.0000,	0.9289,	-1.2321])
theta = theta_deg * np.pi / 180

v = Ht 
k = (Ct - C1) / np.linalg.norm(Ct - C1)
# Rodrigues' rotation formula https://en.wikipedia.org/wiki/Rodrigues%27_rotation_formula
v_rot = v * np.cos(theta) + np.cross(k, v) * np.sin(theta) + k*(np.dot(k,v))*(1.0-np.cos(theta))
print(v_rot)
v_H = np.array([-v_rot[0], -v_rot[1], v_rot[2]])
mol = gto.Mole()
mol.atom = [
['C',   [0.0000,	0.0000,	0.6695]],
['C',   [0.0000,	0.0000,	-0.6695]],
['H',   v_rot],
['H',   v_H],
['H',   [0.0000,	0.9289,	-1.2321]],
['H',   [0.0000,	-0.9289,	-1.2321]]
] 
mol.basis = 'sto-3g'
mol.charge = 0
mol.build()
mf = scf.RHF(mol).run()

In [ ]:
import pandas as pd
table = pd.DataFrame({"Energy": mf.mo_energy, "Occupancy": mf.mo_occ})
table

In [ ]:
def find_homo_lumo(mf):
    lumo = float("inf")
    lumo_idx = None
    homo = -float("inf")
    homo_idx = None
    for i, (energy, occ) in enumerate(zip(mf.mo_energy, mf.mo_occ)):
        if occ > 0 and energy > homo:
            homo = energy
            homo_idx = i
        if occ == 0 and energy < lumo:
            lumo = energy
            lumo_idx = i

    return homo, homo_idx, lumo, lumo_idx


_, homo_idx, _, lumo_idx = find_homo_lumo(mf)
print(f"HOMO (index): {homo_idx}")
print(f"LUMO (index): {lumo_idx}")

In [ ]:
def get_mo(mf, mol):
    """Get molecular orbitals"""
    orbitals = {"canonical": mf.mo_coeff}

    # Get intrinsic bonding orbitals and localized intrinsic valence virtual orbitals (livvo):
    orbocc = mf.mo_coeff[:, 0 : mol.nelec[0]]
    orbvirt = mf.mo_coeff[:, mol.nelec[0] :]

    ovlpS = mol.intor_symmetric("int1e_ovlp")

    iaos = lo.iao.iao(mol, orbocc)
    iaos = lo.orth.vec_lowdin(iaos, ovlpS)
    ibos = lo.ibo.ibo(mol, orbocc, locmethod="IBO")
    orbitals["ibo"] = ibos

    livvo = lo.vvo.livvo(mol, orbocc, orbvirt)
    orbitals["livvo"] = livvo
    return orbitals

In [ ]:
orbitals = get_mo(mf, mol)

In [ ]:
from pyscf import gto, scf, lo, tools
tools.cubegen.orbital(
    mol, "cmo_7.cube", orbitals["canonical"][:, 7], margin=5
)
tools.cubegen.orbital(
    mol, "cmo_8.cube", orbitals["canonical"][:, 8], margin=5
)

tools.cubegen.orbital(
    mol, "cmo_5.cube", orbitals["canonical"][:, 5], margin=5
)
tools.cubegen.orbital(
    mol, "cmo_13.cube", orbitals["canonical"][:, 13], margin=5
)

tools.cubegen.orbital(
    mol, "cmo_4.cube", orbitals["canonical"][:, 4], margin=5
)
tools.cubegen.orbital(
    mol, "cmo_9.cube", orbitals["canonical"][:, 9], margin=5
)

In [ ]:
data = None

cmo_file = "cmo_7.cube"  # use the cube file names specified in previous cell to visualise other orbitals shown in Fig. C.1.

with open(cmo_file, "r") as infile:
    data = infile.read()

In [ ]:
import py3Dmol

view = py3Dmol.view()
view.addVolumetricData(
    data,
    "cube",
    {
        "isoval": 0.05,
        "smoothness": 5,
        "opacity": 0.8,
        "volformat": "cube",
        "color": "blue",
    },
)
view.addVolumetricData(
    data,
    "cube",
    {
        "isoval": -0.05,
        "smoothness": 5,
        "opacity": 0.8,
        "volformat": "cube",
        "color": "orange",
    },
)
view.addModel(data, "cube")
view.rotate(-90, "y")
view.rotate(45, "z")
view.setStyle({"stick": {}})
view.zoomTo()
view.show()

In [ ]:
view.png()